In [1]:
from collections import Counter
import requests
import io
from PIL import Image
import requests
from io import BytesIO
from tika import parser
import pandas as pd
import numpy as np
import urllib
import pyodbc
import sqlalchemy as sa
import math
import fitz
import re, sys, unicodedata
import csv
import gzip
import collections
import xml.etree.ElementTree as ET
from unidecode import unidecode
import os
from PyPDF2 import PdfFileMerger
import urllib.request
import pytesseract as tess
from PIL import Image
stripJunk = str.maketrans("","","- ")

In [2]:
df = pd.read_excel(r'D:\SE\TEST.xlsx',dtype=object)
df = df.replace({ '–' : ' ', '-' : ' '}, regex=True)
data = df['Group_SideEffect_Name']
data

0                          ác mộng
1                      aceton niệu
2                  acid folic giảm
3                     albumin niệu
4               ám điểm ngoại biên
                   ...            
2780    ý tưởng tự gây thương tích
2781             ý thức bất thường
2782                        yếu cơ
2783                       yếu mệt
2784                        yếu ớt
Name: Group_SideEffect_Name, Length: 2785, dtype: object

In [3]:
def getRatio(a,b):
    a = a.lower().translate(stripJunk)
    b = b.lower().translate(stripJunk)
    total  = len(a)+len(b)
    counts = (Counter(a)-Counter(b))+(Counter(b)-Counter(a))
    return 100 - 100 * sum(counts.values()) / total
def dedup(value):
    words = set(value.split(' '))
    return ' '.join(words)

In [16]:
treshold     = 75
minGroupSize = 1

from itertools import combinations

paired = { c:{c} for c in data }
for a,b in combinations(data,2):
    if getRatio(a,b) < treshold: continue
    paired[a].add(b)
    paired[b].add(a)

groups    = list()
ungrouped = set(data)
while ungrouped:
    bestGroup = {}
    for city in ungrouped:
        g = paired[city] & ungrouped
        for c in g.copy():
            g &= paired[c] 
        if len(g) > len(bestGroup):
            bestGroup = g
    if len(bestGroup) < minGroupSize : break  # to terminate grouping early change minGroupSize to 3
    ungrouped -= bestGroup
    groups.append(bestGroup)

In [17]:
result = pd.DataFrame(groups)

In [20]:
result

,0,1,2,3,4,5,6
0,rối loạn chức năng nhận thức,rối loạn chức năng tim,rối loạn chức năng,rối loạn chức năng nút xoang,rối loạn chức năng não,rối loạn chức năng gan,rối loạn chức năng thận
1,tăng nồng độ fibrinogen huyết thanh,tăng cao nồng độ transaminase trong huyết thanh,tăng nồng độ triglycerid huyết thanh,tăng nồng độ các enzym tim trong huyết thanh,tăng nồng độ creatinin huyết thanh,None,None
2,bất thường chức năng gan,kiểm tra chức năng gan bất thường,xét nghiệm chức năng gan bất thường,bất thường chức năng thận,chức năng gan bất bình thường,None,None
3,huyết khối tĩnh mạch thái dương,huyết khối nghẽn mạch,huyết khối tĩnh mạch,huyết khối tĩnh mạch thận,huyết khối nghẽn mạch,None,None
4,khó chịu ở ngực,khó chịu ở họng,khó chịu ở miệng,người khó chịu,khó chịu ở bụng,None,None
...,...,...,...,...,...,...,...
1815,ợ nóng,None,None,None,None,None,None
1816,đổi màu răng,None,None,None,None,None,None
1817,thiếu máu đại hồng cầu,None,None,None,None,None,None
1818,chứng khó đọc,None,None,None,None,None,None


In [21]:
result = pd.DataFrame(groups)
result.fillna("ức chế tủy xương nặngaa",inplace=True)
result = result.replace('   ','  ')

In [22]:
result = result.replace('  ',' ')
# result['concatenated'] = result.apply(lambda x: ','.join(x.astype(str)), axis=1)
result

,0,1,2,3,4,5,6
0,rối loạn chức năng nhận thức,rối loạn chức năng tim,rối loạn chức năng,rối loạn chức năng nút xoang,rối loạn chức năng não,rối loạn chức năng gan,rối loạn chức năng thận
1,tăng nồng độ fibrinogen huyết thanh,tăng cao nồng độ transaminase trong huyết thanh,tăng nồng độ triglycerid huyết thanh,tăng nồng độ các enzym tim trong huyết thanh,tăng nồng độ creatinin huyết thanh,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
2,bất thường chức năng gan,kiểm tra chức năng gan bất thường,xét nghiệm chức năng gan bất thường,bất thường chức năng thận,chức năng gan bất bình thường,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
3,huyết khối tĩnh mạch thái dương,huyết khối nghẽn mạch,huyết khối tĩnh mạch,huyết khối tĩnh mạch thận,huyết khối nghẽn mạch,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
4,khó chịu ở ngực,khó chịu ở họng,khó chịu ở miệng,người khó chịu,khó chịu ở bụng,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
...,...,...,...,...,...,...,...
1815,ợ nóng,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
1816,đổi màu răng,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
1817,thiếu máu đại hồng cầu,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa
1818,chứng khó đọc,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa,ức chế tủy xương nặngaa


In [32]:
# DEDUP_ = []
# for i in range(len(result)):
#     DEDUP = dedup(result['concatenated'][i]).strip().replace('  ',' ')
#     DEDUP_.append(DEDUP)

In [35]:
# result['dedup'] = pd.DataFrame(DEDUP_)

In [23]:
result['min_group'] = result.min(axis=1)
result = result.replace('ức chế tủy xương nặngaa','')
result

,0,1,2,3,4,5,6,min_group
0,rối loạn chức năng nhận thức,rối loạn chức năng tim,rối loạn chức năng,rối loạn chức năng nút xoang,rối loạn chức năng não,rối loạn chức năng gan,rối loạn chức năng thận,rối loạn chức năng
1,tăng nồng độ fibrinogen huyết thanh,tăng cao nồng độ transaminase trong huyết thanh,tăng nồng độ triglycerid huyết thanh,tăng nồng độ các enzym tim trong huyết thanh,tăng nồng độ creatinin huyết thanh,,,tăng cao nồng độ transaminase trong huyết thanh
2,bất thường chức năng gan,kiểm tra chức năng gan bất thường,xét nghiệm chức năng gan bất thường,bất thường chức năng thận,chức năng gan bất bình thường,,,bất thường chức năng gan
3,huyết khối tĩnh mạch thái dương,huyết khối nghẽn mạch,huyết khối tĩnh mạch,huyết khối tĩnh mạch thận,huyết khối nghẽn mạch,,,huyết khối nghẽn mạch
4,khó chịu ở ngực,khó chịu ở họng,khó chịu ở miệng,người khó chịu,khó chịu ở bụng,,,khó chịu ở bụng
...,...,...,...,...,...,...,...,...
1815,ợ nóng,,,,,,,ợ nóng
1816,đổi màu răng,,,,,,,đổi màu răng
1817,thiếu máu đại hồng cầu,,,,,,,thiếu máu đại hồng cầu
1818,chứng khó đọc,,,,,,,chứng khó đọc


In [24]:
result.to_excel(r'D:\SE\SE_GROUP.xlsx')